In [17]:
import numpy as np
import torch
from dolfin import *

/home/haltmayermarc/anaconda3/envs/fenics/lib/python3.11/site-packages/dolfin/jit/jit.py:121: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  def compile_class(cpp_data, mpi_comm=MPI.comm_world):


In [8]:
def u_ini_0(m0, n0, grid):
  grid=torch.tensor(grid)
  return -5.0+m0*torch.sin(n0*grid[:,0])*torch.sin(grid[:,1])

def u_ini_1(m1, n1, grid):
  grid=torch.tensor(grid)
  return 0.0+m1*torch.cos(n1*grid[:,0])*torch.sin(grid[:,1])

In [43]:
mesh = np.load("/home/haltmayermarc/FEONet_Stokes/FEONet_time_dep_Stokes/data_ordered/P2x1_ne200_stokes_zero_BC_dt_0_1.npz", allow_pickle=True)
fenics_u1 = mesh["train_fenics_u1"]
fenics_u2 = mesh["train_fenics_u2"]
fenics_p = mesh["train_fenics_p"]
count = 0
index_list = []
for i in range(fenics_u1.shape[0]):
    if np.isnan(fenics_u1[i,:,:]).any() or np.isinf(fenics_u1[i,:,:]).any():
        index_list.append(i)
        count+=1

In [44]:
count

0

In [48]:
train_coeffs_init = mesh["train_coeffs_init"][34][0]

In [49]:
train_coeffs_init.shape

(441,)

In [55]:
num_element, num_pts, p = mesh['ne'], mesh['ng'], mesh['p']
idx_sol = mesh['idx_sol']
u_solve_linalg=torch.zeros(len(idx_sol[0])+len(idx_sol[1])+len(idx_sol[2]), dtype=torch.float64)
u_solve_linalg[idx_sol[0]]=torch.tensor(mesh['train_coeffs_init'][23][0], dtype=torch.float64)
u_solve_linalg[idx_sol[1]]=torch.tensor(mesh['train_coeffs_init'][23][1], dtype=torch.float64)

In [56]:
u_solve_linalg

tensor([ 0.0142,  0.1162, -0.0750,  ...,  0.0180, -0.0573,  0.0000],
       dtype=torch.float64)

In [28]:
u_init = Expression(( "-5.0+m0*sin(n0*x[0])*sin(x[1])", "0.0+m1*cos(n1*x[0])*sin(x[1])" ), degree=5, m0=m0, m1=m1, n0=n0, n1=n1 )
mesh = RectangleMesh(Point(0, 0), Point(1,1), 10, 10)

# Define the function spaces:
V = VectorElement('CG', triangle, 2)
Q = FiniteElement('CG', triangle, 1)
TH = V * Q
W = FunctionSpace(mesh, TH)

# FENiCS ordering
pos_u1=W.sub(0).sub(0).collapse().tabulate_dof_coordinates()
pos_u2=W.sub(0).sub(1).collapse().tabulate_dof_coordinates()
pos_p=W.sub(1).collapse().tabulate_dof_coordinates()

# numerical ordering
pos_all=W.tabulate_dof_coordinates()
idx_u1=W.sub(0).sub(0).dofmap().dofs()
idx_u2=W.sub(0).sub(1).dofmap().dofs()
idx_p=W.sub(1).dofmap().dofs()

u_b = project(u_init, W.sub ( 0 ).collapse())
u_b_vector = u_b.vector()[:]

In [29]:
u_b_vector_x = u_b.sub(0, deepcopy=True).vector()[:]
u_b_vector_y = u_b.sub(1, deepcopy=True).vector()[:]

In [33]:
u_b_vector_x

array([-5.00004372, -5.00010047, -4.75776441, -4.88270406, -4.87864573,
       -5.00002548, -5.00010803, -4.77457243, -4.89166341, -4.88703825,
       -5.00002939, -4.76591168, -4.51964412, -4.65008842, -4.63800608,
       -5.00010208, -4.79357024, -4.90170579, -4.89655582, -5.00002672,
       -4.78379656, -4.55291677, -4.67681767, -4.66302752, -4.53575285,
       -4.28963304, -4.42336806, -4.40346816, -5.00009235, -4.81462077,
       -4.91273028, -4.90710344, -5.00002433, -4.80383836, -4.59058291,
       -4.70677514, -4.69141153, -4.57121892, -4.33880443, -4.46741704,
       -4.44469453, -4.31341973, -4.07159264, -4.2063621 , -4.17897874,
       -5.00008115, -4.83752171, -4.9246267 , -4.91857852, -5.00002185,
       -4.82584037, -4.63232914, -4.73966275, -4.722875  , -4.61096613,
       -4.39450335, -4.5167854 , -4.49146633, -4.36586938, -4.13583298,
       -4.26698824, -4.23571606, -4.10265398, -3.86919331, -4.00272624,
       -3.96832046, -5.00006904, -4.86204573, -4.93727621, -4.93

In [34]:
u_vector_self_assigned = np.array(u_ini_0(m0 ,n0 ,pos_u1))

/tmp/ipykernel_2095355/1212772415.py:1: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  u_vector_self_assigned = np.array(u_ini_0(m0 ,n0 ,pos_u1))


In [57]:
data = np.load("/home/haltmayermarc/FEONet_Stokes/FEONet_time_dep_Stokes/data_ordered/train/1000N200.pkl", allow_pickle=True)

In [65]:
np.isnan(data[70,0]).any()

np.False_

In [66]:
count = 0
for i in range(1000):
    if np.isnan(data[i,0]).any() or np.isinf(data[i,0]).any():
        count +=1